In [3]:
# For a given n find integers y such that n+y^2 is a square
# Provided by "PM 2Ring" at:
#   MSE: https://math.stackexchange.com/questions/4367306/for-a-perfect-square-n-can-we-calculate-integers-0yl-up-to-a-limit-l-su
#   SageMathCell: https://sagecell.sagemath.org/?z=eJyFUttqwzAMfc9XiMLATt11yegeCin7j9GHJHWYWX2pY1Par58U95KUlRkC8blIyZE6bzWoIH2wdt-D0s76AK2NJghov2tlBDhvd7ENWUfaLpp2ovUSSZlI66Svg_VXTsd9lu1kB66r0eV7pvk6Azw2BqjgaztcOnQ4UCY1ZKwU8M4FsDgwamDoi9hKwAcfwEggUwLUvOT8UpSOwma5gw3oO0an8bL-Gas0vGDTqoK3qVDDclmBm2AHeEScPUrf0y8UGNB2wh2_1V4-rf-sR-qT_wWnZq-1c9Ls2IFPBJjklUm6RA9_uIHi3nwkpI_W2yT0MkRviM2yT2VwFXBUw9CC7APTVVmuLvlS8IGCv2wEy0dzHc2gwVzSWjDcAAG4SgUfZ99Ajs-mehxSa01QJsobWGMlCguasZ3VsICGY8AlBVz8U-Qs4IR10DUnF1bDBbvWoNtN6TwGwGoBjYDZYjNDoyD7bI2vGu2nnID8zH8BSO7LOQ==&lang=python 
from itertools import count, chain, product
from functools import reduce
from operator import mul

def pfactors(m):
    out = []
    for p in chain((2, 3), (u for i in count(5, 6) for u in (i, i+2))):
        if p*p > m:
            break
        if m % p == 0:
            m //= p
            q = p
            powers = [1, p]
            while m % p == 0:
                m //= p
                q *= p
                powers.append(q)
            out.append(powers)
    if m > 1:
        out.append([1, m])
    return out

def test(m=225):
    for t in product(*pfactors(m)):
        b = reduce(mul, t, 1)
        if b * b >= m:
            continue
        a = m // b
        if (a - b) % 2 == 1:
            continue
        z, y = (a + b) // 2, (a - b) // 2
        print(a, b, "->", y, z, ":", m + y*y, z*z)

test()

225 1 -> 112 113 : 12769 12769
45 5 -> 20 25 : 625 625
75 3 -> 36 39 : 1521 1521
25 9 -> 8 17 : 289 289


In [6]:
import math, numpy as np

def is_square(x):
    assert x >= 0
    root = np.uint64(math.sqrt(np.float64(x)) + 0.5)
    return bool(root * root == x), int(root)
    
m = {}
m2 = {}
limit = 1 << 6

for N in range(3, limit, 2):
    l = []
    #N = N*N
    for y in range(1, limit):
        if is_square(N + y * y)[0]:
            l.append(y)
    m[N] = l
    m2[len(l)] = m2.get(len(l), 0) + 1
print(m)
print()
print(dict(sorted(m2.items())))

{3: [1], 5: [2], 7: [3], 9: [4], 11: [5], 13: [6], 15: [1, 7], 17: [8], 19: [9], 21: [2, 10], 23: [11], 25: [12], 27: [3, 13], 29: [14], 31: [15], 33: [4, 16], 35: [1, 17], 37: [18], 39: [5, 19], 41: [20], 43: [21], 45: [2, 6, 22], 47: [23], 49: [24], 51: [7, 25], 53: [26], 55: [3, 27], 57: [8, 28], 59: [29], 61: [30], 63: [1, 9, 31]}

{1: 20, 2: 9, 3: 2}


In [1]:
# Arty's Solution from https://stackoverflow.com/a/70878759/17773824

def find_slow(N):
    import math
    
    def is_square(x):
        root = int(math.sqrt(float(x)) + 0.5)
        return root * root == x, root
    
    l = []
    for y in range(N):
        if is_square(N + y ** 2)[0]:
            l.append(y)
    
    return l
    
def find_fast(N):
    import itertools, functools
    
    Prod = lambda it: functools.reduce(lambda a, b: a * b, it, 1)
    
    fs = factor(N)
    mfs = {}
    for e in fs:
        mfs[e] = mfs.get(e, 0) + 1
    fs = sorted(mfs.items())
    del mfs
    
    Ys = set()
    for take_a in itertools.product(*[
        (range(v + 1) if k != 2 else range(1, v)) for k, v in fs]):
        A = Prod([p ** t for (p, _), t in zip(fs, take_a)])
        B = N // A
        assert A * B == N, (N, A, B, take_a)
        if A < B:
            continue
        X = (A + B) // 2
        Y = (A - B) // 2
        assert N + Y ** 2 == X ** 2, (N, A, B, X, Y)
        Ys.add(Y)
    
    return sorted(Ys)

def trial_div_factor(n, limit = None):
    # https://en.wikipedia.org/wiki/Trial_division
    fs = []
    while n & 1 == 0:
        fs.append(2)
        n >>= 1
    all_checked = False
    for d in range(3, (limit or n) + 1, 2):
        if d * d > n:
            all_checked = True
            break
        while True:
            q, r = divmod(n, d)
            if r != 0:
                break
            fs.append(d)
            n = q
    if n > 1 and all_checked:
        fs.append(n)
        n = 1
    return fs, n

def fermat_prp(n, trials = 32):
    # https://en.wikipedia.org/wiki/Fermat_primality_test
    import random
    if n <= 16:
        return n in (2, 3, 5, 7, 11, 13)
    for i in range(trials):
        if pow(random.randint(2, n - 2), n - 1, n) != 1:
            return False
    return True

def pollard_rho_factor(n):
    # https://en.wikipedia.org/wiki/Pollard%27s_rho_algorithm
    import math, random
    fs, n = trial_div_factor(n, 1 << 7)
    if n <= 1:
        return fs
    if fermat_prp(n):
        return sorted(fs + [n])
    for itry in range(8):
        failed = False
        x = random.randint(2, n - 2)
        for cycle in range(1, 1 << 60):
            y = x
            for i in range(1 << cycle):
                x = (x * x + 1) % n
                d = math.gcd(x - y, n)
                if d == 1:
                    continue
                if d == n:
                    failed = True
                    break
                return sorted(fs + pollard_rho_factor(d) + pollard_rho_factor(n // d))
            if failed:
                break
    assert False, f'Pollard Rho failed! n = {n}'
    
def factor(N):
    import functools
    Prod = lambda it: functools.reduce(lambda a, b: a * b, it, 1)
    fs = pollard_rho_factor(N)
    assert N == Prod(fs), (N, fs)
    return sorted(fs)

def test():
    import random, time
    limit = 1 << 20
    num_tests = 20
    
    t0, t1 = 0, 0
    for i in range(num_tests):
        if (round(i / num_tests * 1000)) % 100 == 0 or i + 1 >= num_tests:
            print(f'test {i}, ', end = '', flush = True)
        
        N = random.randrange(limit)
        
        tb = time.time()
        r0 = find_slow(N)
        t0 += time.time() - tb
        
        tb = time.time()
        r1 = find_fast(N)
        t1 += time.time() - tb
        
        assert r0 == r1, (N, r0, r1, t0, t1)
        
    print(f'\nTime slow {t0:.05f} sec, fast {t1:.05f} sec, speedup {round(t0 / max(1e-6, t1))} times')

if __name__ == '__main__':
    test()

test 0, test 2, test 4, test 6, test 8, test 10, test 12, test 14, test 16, test 18, test 19, 
Time slow 24.46189 sec, fast 0.00468 sec, speedup 5227 times
